In [ ]:
%load_ext sql

# put a folder and DB credential files at HOME directory

import os
homedir = os.getcwd()
cred_path = os.path.join(homedir, 'db_cred')



# add a 'cred_path' for interpreter to search
import sys
sys.path.append(cred_path)



# import DB credentials from 'gpdb_credentials.py' dictionary file.
from gpdb_credentials import dvdrental_db



# parsing DB credentials and connect to Greenplum using %sql $connection_string

username = dvdrental_db['Username']
password = dvdrental_db['Password']
host = dvdrental_db['Host']
port = dvdrental_db['Port']
database = dvdrental_db['Database']

connection_string = 'postgresql://{user}:{password}@{host}:{port}/{db}'.format(
user=username,
password=password,
host=host,
port=port,
db=database)

%sql $connection_string

# HAVING 절

- GROUP BY절에 함께 사용된다.
- GROUP BY절의 결과를 특정 조건으로 `필터링` 하는 역할을 한다.

<HAVING절 문법 예시>

```PYTHON
SELECT
    COLUMN_1, -- GROUP BY에 들어가는 컬럼 
    집계함수(COLUMN_2) 
FROM
    TABLE_NAME
GROUP BY 
    COLUMN_1
HAVING 조건;
```

- HAVING절 VS WHERE절
    - HAVING 절은 GROUP BY절에 의해 생성된 그룹내 데이터 에서 조건을 부여해 필터링 한다.
    - WHERE절은 GROUP BY 절이 적용되기 전에 개별 행(FROM절에서 불러오는 테이블의 모든 행)에 조건을 부여해 필터링 한다.

<참조>
> SQL문 실행순서 (FROM > WHERE > GROUP BY > HAVING > SELECT > ORDER BY)

# 예제 1. GROUP BY + HAVING 

> CUSTOMER_ID 기준으로 판매금액(AMOUNT)를 출력하는데 AMOUNT가 200 이상인 데이터만 출력하시오.

In [ ]:
%%sql

select
	CUSTOMER_ID,
	SUM(AMOUNT) as AMOUNT -- CUSTOMER_ID기준 AMOUNT 합계(SUM)
from 
	PAYMENT
group by
	CUSTOMER_ID
having -- GROUP BY절로 출력된 데이터 내에서 조건 부여하여 필터링
	SUM(AMOUNT) > 200;

## 예제 1-1.  AMOUNT 200 이상인 고객의 이메일 주소 뽑기

> 결제금액(AMOUNT) 200 이상인 고객을 VIP로 선정하고, 이벤트를 진행하고자 한다. 이벤트 정보 발송을 위해 기존 데이터(예제1)에 해당 고객들의 EMAIL정보를 추가하여 출력하시오.

In [ ]:
%%sql

select
	P.CUSTOMER_ID,
	C.EMAIL,
	SUM(P.AMOUNT) as AMOUNT -- CUSTOMER_ID기준 AMOUNT 합계(SUM)
from 
	PAYMENT P,
	CUSTOMER C
where
	P.CUSTOMER_ID = C.CUSTOMER_ID
group by
	P.CUSTOMER_ID,
	C.EMAIL
having -- GROUP BY절로 출력된 데이터 내에서 조건 부여하여 필터링
	SUM(P.AMOUNT) > 200;

# 예제 3. GROUP BY + HAVING (Cont.)

> 고객수가 300명이 넘는 매장의 id와 고객수를 출력하시오.

> 일단 매장 id를 기준으로 고객수를 집계하는 함수를 작성한다.

In [ ]:
%%sql

select 
	STORE_ID,
	COUNT(CUSTOMER_ID) as CX_COUNT -- store기준 customer_id를 count한다.
from
	CUSTOMER
group by
	STORE_ID;

> 위의 쿼리를 기준으로 having절에  count한 고객의 수가 300명 이상인 데이터만 추출한다.

In [ ]:
%%sql

select 
	C.STORE_ID,
	COUNT(C.CUSTOMER_ID) as CX_COUNT
from
	CUSTOMER C
group by
	STORE_ID
having
	COUNT(C.CUSTOMER_ID) > 300;